In [2]:
pip install duckdb

   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.3 MB 2.1 MB/s eta 0:00:06
   --- ------------------------------------ 1.0/12.3 MB 2.1 MB/s eta 0:00:06
   ----- ---------------------------------- 1.6/12.3 MB 2.5 MB/s eta 0:00:05
   ------- -------------------------------- 2.4/12.3 MB 2.6 MB/s eta 0:00:04
   ----------- ---------------------------- 3.4/12.3 MB 3.0 MB/s eta 0:00:03
   ------------- -------------------------- 4.2/12.3 MB 3.2 MB/s eta 0:00:03
   ----------------- ---------------------- 5.2/12.3 MB 3.4 MB/s eta 0:00:03
   ------------------- -------------------- 6.0/12.3 MB 3.5 MB/s eta 0:00:02
   --------------------- ------------------ 6.6/12.3 MB 3.3 MB/s eta 0:00:02
   ---------------------- ----------------- 7.1/12.3 MB 3.4 MB/s eta 0:00:02
   -------------------------- ------------- 8.1/12.3 MB 3.4 MB/s eta 0:00:02
   ----------

In [3]:
# ====== CARGA E INSPECCIÓN INICIAL ======
# Archivo único (CSV grande) — DuckDB lee directo del disco sin cargar todo a RAM.


import duckdb, pandas as pd
from pathlib import Path

FILE_PATH = r"C:\Datos-abiertos-Seguridad-y-Convivencia\data\df_combined_all_years_2020_24.csv"
SEPARATOR = ","      # cambia a ";" si tu CSV usa punto y coma
ENCODING  = "utf-8"  # cambia a "latin-1" si ves tildes raras
N_PREVIEW = 10

con = duckdb.connect(database=":memory:")

print("Leyendo:", FILE_PATH)
con.execute(f"""
    CREATE VIEW v_raw AS
    SELECT * FROM read_csv_auto(
        '{FILE_PATH}',
        header=true,
        sep='{SEPARATOR}',
        encoding='{ENCODING}',
        ignore_errors=true
    );
""")

# 1) ESQUEMA (nombre de columna y tipo)
print("=== ESQUEMA (primeras 120 columnas si hay muchas) ===")
schema_df = con.execute("DESCRIBE v_raw;").df()
display(schema_df.head(120))

# 2) PREVIEW (10 filas)
print("=== PREVIEW ===")
preview_df = con.execute(f"SELECT * FROM v_raw LIMIT {N_PREVIEW};").df()
display(preview_df)

# 3) Conteo total de filas
total = con.execute("SELECT COUNT(*) FROM v_raw;").fetchone()[0]
print(f"Total de filas (aprox): {total:,}")

# 4) Top valores de columnas típicas (se muestran solo si existen)
candidatas = [
    "id","id_caso","id_evento",
    "fecha","hora","anio","mes","dia",
    "departamento","municipio","barrio","comuna","cuadrante",
    "delito","subtipo","modalidad",
    "arma_empleada","arma","sexo_victima","edad_victima","fuente"
]
for c in candidatas:
    try:
        df_top = con.execute(f"""
            SELECT {c} AS valor, COUNT(*) AS n
            FROM v_raw
            GROUP BY 1
            ORDER BY n DESC NULLS LAST
            LIMIT 10;
        """).df()
        if len(df_top):
            print(f"\nTop valores de {c}:")
            display(df_top)
    except Exception:
        pass


Leyendo: C:\Datos-abiertos-Seguridad-y-Convivencia\data\df_combined_all_years_2020_24.csv
=== ESQUEMA (primeras 120 columnas si hay muchas) ===


,column_name,column_type,null,key,default,extra
0,ARMAS_MEDIOS,VARCHAR,YES,None,None,None
1,FECHA,TIMESTAMP,YES,None,None,None
2,CANTIDAD,DOUBLE,YES,None,None,None
3,TIPO_DELITO,VARCHAR,YES,None,None,None
4,GENERO,VARCHAR,YES,None,None,None
5,AGRUPA_EDAD_PERSONA,VARCHAR,YES,None,None,None
6,DELITO,VARCHAR,YES,None,None,None
7,Código Departamento,BIGINT,YES,None,None,None
8,Nombre Departamento,VARCHAR,YES,None,None,None
9,Código Municipio,BIGINT,YES,None,None,None


=== PREVIEW ===


,ARMAS_MEDIOS,FECHA,CANTIDAD,TIPO_DELITO,GENERO,AGRUPA_EDAD_PERSONA,DELITO,Código Departamento,Nombre Departamento,Código Municipio,Nombre Municipio,Tipo: Municipio / Isla / Área no municipalizada,longitud,Latitud,AÑO,ÁREA GEOGRÁFICA,TOTAL
0,ARMA DE FUEGO,2020-08-21,1.0,HURTO PIRATERÍA TERRESTRE,None,None,None,5,ANTIOQUIA,5001,MEDELLÍN,Municipio,"-75,581775","6,246631",2020.0,Total,2489955.0
1,ARMA DE FUEGO,2020-08-21,1.0,HURTO PIRATERÍA TERRESTRE,None,None,None,5,ANTIOQUIA,5001,MEDELLÍN,Municipio,"-75,581775","6,246631",2021.0,Total,2507620.0
2,ARMA DE FUEGO,2020-08-21,1.0,HURTO PIRATERÍA TERRESTRE,None,None,None,5,ANTIOQUIA,5001,MEDELLÍN,Municipio,"-75,581775","6,246631",2022.0,Total,2514709.0
3,ARMA DE FUEGO,2020-10-14,1.0,HURTO PIRATERÍA TERRESTRE,None,None,None,5,ANTIOQUIA,5001,MEDELLÍN,Municipio,"-75,581775","6,246631",2020.0,Total,2489955.0
4,ARMA DE FUEGO,2020-10-14,1.0,HURTO PIRATERÍA TERRESTRE,None,None,None,5,ANTIOQUIA,5001,MEDELLÍN,Municipio,"-75,581775","6,246631",2021.0,Total,2507620.0
5,ARMA DE FUEGO,2020-10-14,1.0,HURTO PIRATERÍA TERRESTRE,None,None,None,5,ANTIOQUIA,5001,MEDELLÍN,Municipio,"-75,581775","6,246631",2022.0,Total,2514709.0
6,ARMA DE FUEGO,2020-11-18,2.0,HURTO PIRATERÍA TERRESTRE,None,None,None,5,ANTIOQUIA,5001,MEDELLÍN,Municipio,"-75,581775","6,246631",2020.0,Total,2489955.0
7,ARMA DE FUEGO,2020-11-18,2.0,HURTO PIRATERÍA TERRESTRE,None,None,None,5,ANTIOQUIA,5001,MEDELLÍN,Municipio,"-75,581775","6,246631",2021.0,Total,2507620.0
8,ARMA DE FUEGO,2020-11-18,2.0,HURTO PIRATERÍA TERRESTRE,None,None,None,5,ANTIOQUIA,5001,MEDELLÍN,Municipio,"-75,581775","6,246631",2022.0,Total,2514709.0
9,ARMA DE FUEGO,2020-12-22,1.0,HURTO PIRATERÍA TERRESTRE,None,None,None,5,ANTIOQUIA,5001,MEDELLÍN,Municipio,"-75,581775","6,246631",2020.0,Total,2489955.0


Total de filas (aprox): 5,217,120

Top valores de fecha:


,valor,n
0,2020-01-01,10655
1,2022-01-01,5996
2,2020-03-08,5971
3,2020-03-01,5931
4,2020-02-02,5464
5,2021-01-01,5419
6,2020-01-12,5328
7,2020-01-05,5055
8,2021-08-01,5005
9,2020-02-09,4979



Top valores de delito:


,valor,n
0,None,4855171
1,ARTÍCULO 209. ACTOS SEXUALES CON MENOR DE 14 AÑOS,127289
2,ARTÍCULO 208. ACCESO CARNAL ABUSIVO CON MENOR ...,79160
3,ARTÍCULO 205. ACCESO CARNAL VIOLENTO,51214
4,ARTÍCULO 206. ACTO SEXUAL VIOLENTO,34802
5,ARTÍCULO 210 A. ACOSO SEXUAL,25619
6,ARTÍCULO 210. ACCESO CARNAL O ACTO SEXUAL ABUS...,17151
7,ARTÍCULO 218. PORNOGRAFÍA CON MENORES,11124
8,ARTÍCULO 207. ACCESO CARNAL O ACTO SEXUAL EN P...,5795
9,ARTÍCULO 219 A. UTILIZACIÓN O FACILITACIÓN DE ...,3123


In [6]:
# ============================================================
# CONTEO DE FILAS TOTALES (sin cargar todo en memoria)
# ============================================================

import duckdb

# Reusamos la conexión anterior o creamos una nueva si no existe
try:
    con
except NameError:
    con = duckdb.connect(database=":memory:")

FILE_PATH = r"C:\Datos-abiertos-Seguridad-y-Convivencia\data\df_combined_all_years_2020_24.csv"
SEPARATOR = ","
ENCODING  = "utf-8"

query_count = f"""
SELECT COUNT(*) AS total_filas
FROM read_csv_auto('{FILE_PATH}', header=true, sep='{SEPARATOR}', encoding='{ENCODING}', ignore_errors=true);
"""

total_filas = con.execute(query_count).fetchone()[0]
print(f"📊 Total de filas en la base de delitos: {total_filas:,}")


📊 Total de filas en la base de delitos: 5,217,120


In [7]:
# ===============================================
# ETL a SQLite (robusto para 5.2M filas)
# - Ingesta en chunks con pandas
# - Normalización de nombres
# - Creación de dimensiones y tabla de hechos
# ===============================================

import sqlite3, pandas as pd, numpy as np
from pathlib import Path
from unidecode import unidecode

# ---------- Config ----------
CSV_PATH = r"C:\Datos-abiertos-Seguridad-y-Convivencia\data\df_combined_all_years_2020_24.csv"
DB_PATH  = r"C:\Datos-abiertos-Seguridad-y-Convivencia\modelo_delitos.db"
CHUNK    = 200_000
ENCODING = "utf-8"
SEPARATOR = ","

# ---------- Helpers ----------
def to_snake(s: str) -> str:
    s = unidecode(str(s)).strip()
    # reemplazos básicos
    s = s.replace("/", " ").replace("-", " ").replace("%", " pct ")
    # quitar caracteres raros
    s = "".join(ch if (ch.isalnum() or ch.isspace()) else "_" for ch in s)
    # espacios -> _
    s = "_".join([t for t in s.lower().split() if t])
    # colapsar __
    s = "_".join([t for t in s.split("_") if t])
    return s

# Mapeo manual de nombres -> snake_case canónico
RENAME = {
    "ARMAS_MEDIOS": "armas_medios",
    "FECHA": "fecha",
    "CANTIDAD": "cantidad",
    "TIPO_DELITO": "tipo_delito",
    "GENERO": "genero",
    "AGRUPA_EDAD_PERSONA": "agrupa_edad_persona",
    "DELITO": "delito",
    "Código Departamento": "codigo_departamento",
    "Nombre Departamento": "nombre_departamento",
    "Código Municipio": "codigo_municipio",
    "Nombre Municipio": "nombre_municipio",
    "Tipo: Municipio / Isla / Área no municipalizada": "tipo_area",
    "longitud": "longitud",
    "Latitud": "latitud",
    "AÑO": "anio",
    "ÁREA GEOGRÁFICA": "area_geografica",
    "TOTAL": "total",
}

# ---------- 1) Ingesta incremental a SQLite (tabla raw) ----------
con = sqlite3.connect(DB_PATH)
cur = con.cursor()

cur.execute("PRAGMA journal_mode=WAL;")
cur.execute("PRAGMA synchronous=NORMAL;")
cur.execute("PRAGMA temp_store=MEMORY;")

RAW = "raw_delitos"

# Creamos la tabla vacía con un esquema flexible (la primera vez que entra un chunk)
created = False
total_rows = 0

for i, chunk in enumerate(pd.read_csv(
        CSV_PATH,
        sep=SEPARATOR,
        encoding=ENCODING,
        chunksize=CHUNK,
        low_memory=False,
        dtype=str  # leemos todo como str para evitar sorpresas; convertimos después en SQL
    )):
    # Normalizar columnas
    cols = [RENAME.get(c, c) for c in chunk.columns]
    cols = [to_snake(c) for c in cols]
    chunk.columns = cols

    # Limpiezas rápidas
    if "fecha" in chunk.columns:
        # Guardar fecha como texto ISO; luego la parseamos en SQL si hace falta
        pass
    if "cantidad" in chunk.columns:
        # Si viene como texto, intentar a float; si falla, la dejamos tal cual (SQL la convertirá)
        chunk["cantidad"] = pd.to_numeric(chunk["cantidad"], errors="coerce")
    if "total" in chunk.columns:
        chunk["total"] = pd.to_numeric(chunk["total"], errors="coerce")

    # Insertar
    chunk.to_sql(RAW, con, if_exists="append", index=False)
    total_rows += len(chunk)
    print(f"→ Cargado chunk {i+1:,} (acum: {total_rows:,})")

print(f"✔️ Ingesta completa en SQLite: {total_rows:,} filas en {RAW}")

# ---------- 2) Crear dimensiones ----------
# dim_fecha
cur.execute("DROP TABLE IF EXISTS dim_fecha;")
cur.execute("""
CREATE TABLE dim_fecha AS
SELECT
  ROW_NUMBER() OVER () AS id_fecha,
  fecha,
  CAST(substr(fecha,1,4) AS INT) AS anio,
  CAST(substr(fecha,6,2) AS INT) AS mes,
  CAST(substr(fecha,9,2) AS INT) AS dia
FROM (
  SELECT DISTINCT fecha FROM raw_delitos WHERE fecha IS NOT NULL
)
ORDER BY fecha;
""")

# dim_ubicacion
cur.execute("DROP TABLE IF EXISTS dim_ubicacion;")
cur.execute("""
CREATE TABLE dim_ubicacion AS
SELECT
  ROW_NUMBER() OVER () AS id_ubicacion,
  CAST(codigo_departamento AS TEXT) AS codigo_departamento,
  UPPER(TRIM(nombre_departamento))  AS nombre_departamento,
  CAST(codigo_municipio AS TEXT)    AS codigo_municipio,
  UPPER(TRIM(nombre_municipio))     AS nombre_municipio,
  UPPER(TRIM(tipo_area))            AS tipo_area,
  CAST(REPLACE(REPLACE(longitud, ',', '.'), ' ', '') AS REAL) AS longitud,
  CAST(REPLACE(REPLACE(latitud, ',', '.'),  ' ', '') AS REAL) AS latitud
FROM (
  SELECT DISTINCT
    codigo_departamento, nombre_departamento, codigo_municipio, nombre_municipio,
    tipo_area, longitud, latitud
  FROM raw_delitos
)
ORDER BY nombre_departamento, nombre_municipio;
""")

# dim_delito
cur.execute("DROP TABLE IF EXISTS dim_delito;")
cur.execute("""
CREATE TABLE dim_delito AS
SELECT
  ROW_NUMBER() OVER () AS id_delito,
  UPPER(TRIM(tipo_delito)) AS tipo_delito,
  UPPER(TRIM(delito))      AS delito,
  UPPER(TRIM(armas_medios)) AS armas_medios
FROM (
  SELECT DISTINCT tipo_delito, delito, armas_medios
  FROM raw_delitos
)
ORDER BY tipo_delito, delito, armas_medios;
""")

# dim_victima
cur.execute("DROP TABLE IF EXISTS dim_victima;")
cur.execute("""
CREATE TABLE dim_victima AS
SELECT
  ROW_NUMBER() OVER () AS id_victima,
  UPPER(TRIM(genero)) AS genero,
  UPPER(TRIM(agrupa_edad_persona)) AS agrupa_edad_persona
FROM (
  SELECT DISTINCT genero, agrupa_edad_persona
  FROM raw_delitos
)
ORDER BY genero, agrupa_edad_persona;
""")

con.commit()
print("✔️ Dimensiones creadas: dim_fecha, dim_ubicacion, dim_delito, dim_victima")

# ---------- 3) Índices para acelerar joins ----------
cur.executescript("""
CREATE INDEX IF NOT EXISTS idx_raw_fecha ON raw_delitos(fecha);
CREATE INDEX IF NOT EXISTS idx_raw_ubicacion ON raw_delitos(codigo_departamento, nombre_departamento, codigo_municipio, nombre_municipio);
CREATE INDEX IF NOT EXISTS idx_raw_delito ON raw_delitos(tipo_delito, delito, armas_medios);
CREATE INDEX IF NOT EXISTS idx_raw_victima ON raw_delitos(genero, agrupa_edad_persona);

CREATE INDEX IF NOT EXISTS idx_dim_fecha_fecha ON dim_fecha(fecha);
CREATE INDEX IF NOT EXISTS idx_dim_ubicacion_keys ON dim_ubicacion(codigo_departamento, nombre_departamento, codigo_municipio, nombre_municipio);
CREATE INDEX IF NOT EXISTS idx_dim_delito_keys ON dim_delito(tipo_delito, delito, armas_medios);
CREATE INDEX IF NOT EXISTS idx_dim_victima_keys ON dim_victima(genero, agrupa_edad_persona);
""")
con.commit()

# ---------- 4) Tabla de hechos ----------
cur.execute("DROP TABLE IF EXISTS fact_crime;")
cur.execute("""
CREATE TABLE fact_crime AS
SELECT
  f.id_fecha,
  u.id_ubicacion,
  d.id_delito,
  v.id_victima,
  COALESCE(r.cantidad, 1) AS cantidad,
  r.total
FROM raw_delitos r
LEFT JOIN dim_fecha     f ON f.fecha = r.fecha
LEFT JOIN dim_ubicacion u ON
     u.codigo_departamento = CAST(r.codigo_departamento AS TEXT)
 AND u.nombre_departamento = UPPER(TRIM(r.nombre_departamento))
 AND u.codigo_municipio    = CAST(r.codigo_municipio AS TEXT)
 AND u.nombre_municipio    = UPPER(TRIM(r.nombre_municipio))
LEFT JOIN dim_delito    d ON
     d.tipo_delito  = UPPER(TRIM(r.tipo_delito))
 AND d.delito       = UPPER(TRIM(r.delito))
 AND d.armas_medios = UPPER(TRIM(r.armas_medios))
LEFT JOIN dim_victima   v ON
     v.genero              = UPPER(TRIM(r.genero))
 AND v.agrupa_edad_persona = UPPER(TRIM(r.agrupa_edad_persona));
""")
con.commit()

# ---------- 5) Chequeos rápidos ----------
# conteos deben cuadrar
staging_rows = cur.execute("SELECT COUNT(*) FROM raw_delitos;").fetchone()[0]
fact_rows    = cur.execute("SELECT COUNT(*) FROM fact_crime;").fetchone()[0]
null_keys    = cur.execute("""
SELECT
  SUM(CASE WHEN id_fecha     IS NULL THEN 1 ELSE 0 END),
  SUM(CASE WHEN id_ubicacion IS NULL THEN 1 ELSE 0 END),
  SUM(CASE WHEN id_delito    IS NULL THEN 1 ELSE 0 END),
  SUM(CASE WHEN id_victima   IS NULL THEN 1 ELSE 0 END)
FROM fact_crime;
""").fetchone()

print(f"Filas en raw_delitos: {staging_rows:,}")
print(f"Filas en fact_crime : {fact_rows:,}")
print(f"Null keys (fecha, ubicacion, delito, victima): {null_keys}")

con.close()
print(f"✅ Modelo estrella creado en: {DB_PATH}")


→ Cargado chunk 1 (acum: 200,000)
→ Cargado chunk 2 (acum: 400,000)
→ Cargado chunk 3 (acum: 600,000)
→ Cargado chunk 4 (acum: 800,000)
→ Cargado chunk 5 (acum: 1,000,000)
→ Cargado chunk 6 (acum: 1,200,000)
→ Cargado chunk 7 (acum: 1,400,000)
→ Cargado chunk 8 (acum: 1,600,000)
→ Cargado chunk 9 (acum: 1,800,000)
→ Cargado chunk 10 (acum: 2,000,000)
→ Cargado chunk 11 (acum: 2,200,000)
→ Cargado chunk 12 (acum: 2,400,000)
→ Cargado chunk 13 (acum: 2,600,000)
→ Cargado chunk 14 (acum: 2,800,000)
→ Cargado chunk 15 (acum: 3,000,000)
→ Cargado chunk 16 (acum: 3,200,000)
→ Cargado chunk 17 (acum: 3,400,000)
→ Cargado chunk 18 (acum: 3,600,000)
→ Cargado chunk 19 (acum: 3,800,000)
→ Cargado chunk 20 (acum: 4,000,000)
→ Cargado chunk 21 (acum: 4,200,000)
→ Cargado chunk 22 (acum: 4,400,000)
→ Cargado chunk 23 (acum: 4,600,000)
→ Cargado chunk 24 (acum: 4,800,000)
→ Cargado chunk 25 (acum: 5,000,000)
→ Cargado chunk 26 (acum: 5,200,000)
→ Cargado chunk 27 (acum: 5,217,120)
✔️ Ingesta complet

In [10]:
import sqlite3

# Conectarse a la base de datos donde ya cargaste los datos
DB_PATH = r"C:\Datos-abiertos-Seguridad-y-Convivencia\modelo_delitos.db"
con = sqlite3.connect(DB_PATH)
cur = con.cursor()


In [11]:
# dim_fecha
cur.execute("DROP TABLE IF EXISTS dim_fecha;")
cur.execute("""
CREATE TABLE dim_fecha AS
SELECT
  ROW_NUMBER() OVER () AS id_fecha,
  fecha,
  CAST(substr(fecha,1,4) AS INT) AS anio,
  CAST(substr(fecha,6,2) AS INT) AS mes,
  CAST(substr(fecha,9,2) AS INT) AS dia
FROM (
  SELECT DISTINCT fecha FROM raw_delitos WHERE fecha IS NOT NULL
)
ORDER BY fecha;
""")

In [12]:
# dim_ubicacion
cur.execute("DROP TABLE IF EXISTS dim_ubicacion;")
cur.execute("""
CREATE TABLE dim_ubicacion AS
SELECT
  ROW_NUMBER() OVER () AS id_ubicacion,
  CAST(codigo_departamento AS TEXT) AS codigo_departamento,
  UPPER(TRIM(nombre_departamento))  AS nombre_departamento,
  CAST(codigo_municipio AS TEXT)    AS codigo_municipio,
  UPPER(TRIM(nombre_municipio))     AS nombre_municipio,
  UPPER(TRIM(tipo_area))            AS tipo_area,
  CAST(REPLACE(REPLACE(longitud, ',', '.'), ' ', '') AS REAL) AS longitud,
  CAST(REPLACE(REPLACE(latitud, ',', '.'),  ' ', '') AS REAL) AS latitud
FROM (
  SELECT DISTINCT
    codigo_departamento, nombre_departamento, codigo_municipio, nombre_municipio,
    tipo_area, longitud, latitud
  FROM raw_delitos
)
ORDER BY nombre_departamento, nombre_municipio;
""")


In [13]:
# dim_delito
cur.execute("DROP TABLE IF EXISTS dim_delito;")
cur.execute("""
CREATE TABLE dim_delito AS
SELECT
  ROW_NUMBER() OVER () AS id_delito,
  UPPER(TRIM(tipo_delito)) AS tipo_delito,
  UPPER(TRIM(delito))      AS delito,
  UPPER(TRIM(armas_medios)) AS armas_medios
FROM (
  SELECT DISTINCT tipo_delito, delito, armas_medios
  FROM raw_delitos
)
ORDER BY tipo_delito, delito, armas_medios;
""")

In [14]:
# dim_victima
cur.execute("DROP TABLE IF EXISTS dim_victima;")
cur.execute("""
CREATE TABLE dim_victima AS
SELECT
  ROW_NUMBER() OVER () AS id_victima,
  UPPER(TRIM(genero)) AS genero,
  UPPER(TRIM(agrupa_edad_persona)) AS agrupa_edad_persona
FROM (
  SELECT DISTINCT genero, agrupa_edad_persona
  FROM raw_delitos
)
ORDER BY genero, agrupa_edad_persona;
""")

In [15]:
cur.executescript("""
CREATE INDEX IF NOT EXISTS idx_raw_fecha ON raw_delitos(fecha);
CREATE INDEX IF NOT EXISTS idx_raw_ubicacion ON raw_delitos(codigo_departamento, nombre_departamento, codigo_municipio, nombre_municipio);
CREATE INDEX IF NOT EXISTS idx_raw_delito ON raw_delitos(tipo_delito, delito, armas_medios);
CREATE INDEX IF NOT EXISTS idx_raw_victima ON raw_delitos(genero, agrupa_edad_persona);

CREATE INDEX IF NOT EXISTS idx_dim_fecha_fecha ON dim_fecha(fecha);
CREATE INDEX IF NOT EXISTS idx_dim_ubicacion_keys ON dim_ubicacion(codigo_departamento, nombre_departamento, codigo_municipio, nombre_municipio);
CREATE INDEX IF NOT EXISTS idx_dim_delito_keys ON dim_delito(tipo_delito, delito, armas_medios);
CREATE INDEX IF NOT EXISTS idx_dim_victima_keys ON dim_victima(genero, agrupa_edad_persona);
""")
con.commit()

In [16]:
# ---------- 4) Tabla de hechos ----------
cur.execute("DROP TABLE IF EXISTS fact_crime;")
cur.execute("""
CREATE TABLE fact_crime AS
SELECT
  f.id_fecha,
  u.id_ubicacion,
  d.id_delito,
  v.id_victima,
  COALESCE(r.cantidad, 1) AS cantidad,
  r.total
FROM raw_delitos r
LEFT JOIN dim_fecha     f ON f.fecha = r.fecha
LEFT JOIN dim_ubicacion u ON
     u.codigo_departamento = CAST(r.codigo_departamento AS TEXT)
 AND u.nombre_departamento = UPPER(TRIM(r.nombre_departamento))
 AND u.codigo_municipio    = CAST(r.codigo_municipio AS TEXT)
 AND u.nombre_municipio    = UPPER(TRIM(r.nombre_municipio))
LEFT JOIN dim_delito    d ON
     d.tipo_delito  = UPPER(TRIM(r.tipo_delito))
 AND d.delito       = UPPER(TRIM(r.delito))
 AND d.armas_medios = UPPER(TRIM(r.armas_medios))
LEFT JOIN dim_victima   v ON
     v.genero              = UPPER(TRIM(r.genero))
 AND v.agrupa_edad_persona = UPPER(TRIM(r.agrupa_edad_persona));
""")
con.commit()

In [17]:
# conteos deben cuadrar
staging_rows = cur.execute("SELECT COUNT(*) FROM raw_delitos;").fetchone()[0]
fact_rows    = cur.execute("SELECT COUNT(*) FROM fact_crime;").fetchone()[0]
null_keys    = cur.execute("""
SELECT
  SUM(CASE WHEN id_fecha     IS NULL THEN 1 ELSE 0 END),
  SUM(CASE WHEN id_ubicacion IS NULL THEN 1 ELSE 0 END),
  SUM(CASE WHEN id_delito    IS NULL THEN 1 ELSE 0 END),
  SUM(CASE WHEN id_victima   IS NULL THEN 1 ELSE 0 END)
FROM fact_crime;
""").fetchone()

print(f"Filas en raw_delitos: {staging_rows:,}")
print(f"Filas en fact_crime : {fact_rows:,}")
print(f"Null keys (fecha, ubicacion, delito, victima): {null_keys}")

con.close()

Filas en raw_delitos: 5,217,120
Filas en fact_crime : 5,218,731
Null keys (fecha, ubicacion, delito, victima): (228, 0, 4856434, 561126)


testeo con consultas

In [21]:
import sqlite3, pandas as pd

DB_PATH = r"C:\Datos-abiertos-Seguridad-y-Convivencia\modelo_delitos.db"  # ajusta si la guardaste en otra ruta
con = sqlite3.connect(DB_PATH)

# (opcional) mira qué tablas hay para confirmar que estás en la DB correcta
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", con)


,name
0,dim_delito
1,dim_fecha
2,dim_ubicacion
3,dim_victima
4,fact_crime
5,raw_delitos


In [22]:
query = """
SELECT 
    f2.anio,
    COUNT(*) AS total_casos
FROM fact_crime fc
JOIN dim_fecha f2 ON fc.id_fecha = f2.id_fecha
GROUP BY f2.anio
ORDER BY f2.anio;
"""
df = pd.read_sql(query, con)
df


,anio,total_casos
0,2020,1183462
1,2021,1130959
2,2022,1163827
3,2023,840804
4,2024,899451


In [23]:
query = """
SELECT 
    f2.anio,
    d.tipo_delito,
    COUNT(*) AS total_casos
FROM fact_crime fc
JOIN dim_fecha f2 ON fc.id_fecha = f2.id_fecha
JOIN dim_delito d ON fc.id_delito = d.id_delito
GROUP BY f2.anio, d.tipo_delito
ORDER BY f2.anio, total_casos DESC;
"""
df = pd.read_sql(query, con)
df

,anio,tipo_delito,total_casos
0,2020,DELITOS SEXUALES,90985
1,2021,DELITOS SEXUALES,98151
2,2021,SECUESTRO,456
3,2022,DELITOS SEXUALES,73884
4,2022,SECUESTRO,705
5,2023,DELITOS SEXUALES,48278
6,2024,DELITOS SEXUALES,49838


In [25]:
query = """
SELECT 
    f2.anio,
    u.nombre_departamento,
    SUM(fc.cantidad) AS total_casos
FROM fact_crime fc
JOIN dim_fecha f2 ON fc.id_fecha = f2.id_fecha
JOIN dim_ubicacion u ON fc.id_ubicacion = u.id_ubicacion
GROUP BY f2.anio, u.nombre_departamento
ORDER BY f2.anio, total_casos DESC;
"""
df = pd.read_sql(query, con)
df

,anio,nombre_departamento,total_casos
0,2020,ANTIOQUIA,303354.0
1,2020,CUNDINAMARCA,141831.0
2,2020,SANTANDER,130262.0
3,2020,ATLÁNTICO,91728.0
4,2020,VALLE DEL CAUCA,84558.0
...,...,...,...
155,2024,ARAUCA,3756.0
156,2024,GUAINÍA,2118.0
157,2024,AMAZONAS,1344.0
158,2024,VICHADA,878.0


In [26]:
con.close()